# Add a 'time to linearize' output to diffusion program

# Perform a Morris SA on function

In [1]:
%matplotlib inline

from gekko import GEKKO
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation, rc
from IPython.display import HTML, Image

In [2]:
from scipy.stats import linregress

## REAL WORLD PARAMETERS:

### Current device channel dimensions:

Length: 11mm

Height: 1mm

Width: 1mm

### Initial Chemoattractant concentration:

C0 = 100 nanomolar

### Buffer Solution: Basically water

### Chemoattractant protein:

TGF-Beta

Approximate diffusion constant from particle size 

#### 

r = (1.9)*(1/1000000000) #m 
Radius of spherical particle --> https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3055910/ AND https://www.abcam.com/recombinant-human-tgf-beta-1-protein-active-ab50036.html#:~:text=TGF%2Dbeta1%20is%20a%2025.0,peptide%20and%20latency%2Dassociated%20peptide.

D = (k*T)(6pi*mu*r)

Where k is boltzmann constant (1.380649 * 10^-23 J/K), T is temperature (K), mu is dynamic visc of water (.0006922 Pa * s),
and r is particle radius


## Simulation data for existing product IBIDI Mu-Slide Chemotaxis Chamber

### IBIDI device channel dimensions:

Length: 1mm

Width: 2mm

Height: .07mm

### IBIDI Device does not have any nanofibers, disregard adsoprtion

Ibidi device time to linearize = 26 hours

All other parameters (chemoattractant, buffer, C0) should be same

In [3]:
def linearization(linearity = .99, alpha = 5, beta = 1, n = 3):
    ## Inputs
    '''
    # Dimensionless Parameters
    alpha = 5.  # k*L^2/D
    beta = 1.   # f*(S0/C0)^(1/n)
    n = 3.      # n
    '''
    # Boundary conditions
    theta_BC = [1., 0.]

    # Solver parameters
    seg = 40   # Number of length segments
    dt = 0.05   # Time step size
    tf = 1.5  # Final time
    
    L_seg = 1 / seg                     # Length of segment
    dist = np.linspace(0., 1., seg + 2) # Distance array
    nt = int(tf / dt) + 1               # Length of time array
    
    # Create new GEKKO object
    m = GEKKO()

    # Create time array
    m.time = np.linspace(0., tf, nt)

    # Left BC
    theta0 = m.MV(lb = 0., ub = 1.)
    theta0.value = np.ones(nt) * theta_BC[0]

    # Right BC
    theta1 = m.MV(lb = 0., ub = 1.)
    theta1.value = np.ones(nt) * theta_BC[1]

    # Initialize concentration variables
    theta = [m.Var(0., lb = 0., ub = 1.) for i in range(seg)]
    theta_BL = [m.Var(0., lb = 0., ub = 1.) for i in range(seg)]
    # theta_ads = [m.Var(0, lb = 0, ub = 1) for i in range(seg)]
    
    ## Equations to be solved

    # First segment
    m.Equation(theta[0].dt() == -theta_BL[0].dt() + (1 * theta0 + -2 * theta[0] + 1 * theta[1]) / (L_seg * L_seg))
    m.Equation(theta_BL[0].dt() == alpha * (theta[0] - theta_BL[0]))
    # m.Equation(theta_ads[0] == beta * theta_BL[0] ** (1 / n))

    # Middle segments
    m.Equations([theta[i].dt() == -theta_BL[i].dt() + (1 * theta[i - 1] + -2 * theta[i] + 1 * theta[i + 1]) / (L_seg * L_seg) for i in range(1, seg - 1)])
    m.Equations([theta_BL[i].dt() == alpha * (theta[i] - theta_BL[i]) for i in range(1, seg - 1)])
    # m.Equations([theta_ads[i] == beta * theta_BL[i] ** (1 / n) for i in range(1, seg - 1)])

    # Last segment
    m.Equation(theta[-1].dt() == -theta_BL[0].dt() + (1 * theta[-2] + -2 * theta[-1] + 1 * theta1) / (L_seg * L_seg))
    m.Equation(theta_BL[-1].dt() == alpha * (theta[-1] - theta_BL[-1]))
    # m.Equation(theta_ads[-1] == beta * theta_BL[-1] ** (1 / n))
    
    m.options.IMODE = 4
    m.solve()
    
    # Bulk
    d = np.empty((seg + 2, len(m.time)))
    d[0] = np.array(theta0.value)
    for i in range(seg):
        d[i + 1] = np.array(theta[i].value)
    d[-1] = np.array(theta1.value)
    d = d.T
    
    t_Bulk = time_to_linearize(d, dist, linearity)*dt
    
    # Boundary later
    d_BL = np.empty((seg + 1, len(m.time)))
    for i in range(seg):
        d_BL[i] = np.array(theta_BL[i].value)
    d_BL[-1] = np.array(theta1.value)
    d_BL = d_BL.T
    
    t_BL = time_to_linearize(d_BL, dist, linearity)*dt

    # Adsorbed
    d_ads = beta * d_BL ** (1 / n)

    t_ads = time_to_linearize(d_ads, dist, linearity)*dt
    
    return [t_Bulk, t_BL, t_ads]
    
    
    
    
    
    

In [4]:
def time_to_linearize(C_profile, dist_array, desired_linearity):
    '''
    C_profile is list of concentration profile arrays at each time
    
    C_profile = [C_timestamp(t=0), C_timestamp(t=1),..]
    C_timestamp = [C(x=0),C(x=1),....]
    
    dist_array = list of distances. Single 1-D array
    '''
    
    for t_index, C_timestamp in enumerate(C_profile):
        regress_results = linregress(C_timestamp, dist_array)
        linearity = 1 - regress_results.stderr
        if linearity >= desired_linearity:
            return t_index
        else:
            continue
    
    return False
    '''
    Returns time as an integer relative to overall time function/segmentation.
    
    If C_profile is created with 60 time intervals, and this function returns '34':
    That means linearization was achieved at time 34/60 * overall time. 
    
    returns False if desired linearity never achieved
    '''


In [8]:
# linearization()

# Morris Sensitivity Analysis

In [7]:
import SALib

In [9]:
from SALib.sample import morris as ms
from SALib.analyze import morris as ma
from SALib.plotting import morris as mp

In [13]:
morris_problem = {
    'num_vars': 3,
    'names': ['alpha','beta','n'],
    'bounds': [[15, 90],
               [8000,20000],
               [.1, 20]], 
    'groups': None
}

In [12]:
num_levels = 4
trajectories = 1000
sample = ms.sample(morris_problem, trajectories, num_levels = num_levels)
sample.shape

In [ ]:
Si = ma.analyze(morris_problem,
               sample,
               output,
               print_to_console=False,
               num_levels=num_levels)
print("{:20s} {:>7s} {:>7s} {:>7s}".format("Name", "mu", "mu_star", "sigma"))
for name, s1, st, mean in zip(morris_problem['names'], Si['mu'], Si['mu_star'], Si['sigma']):
    print("{:20s} {:=7.2f} {:=7.2f} {:=7.2f}".format(name, s1, st, mean))
    
fig, (ax1, ax2) = plt.subplots(1,2)
mp.horizontal_bar_plot(ax1, Si) #  param_dict={}
mp.covariance_plot(ax2, Si, {})